In [1]:
!pip install openaq


StatementMeta(, 035c5686-36d6-479b-b364-8240bbc7849c, 3, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 22.3 MB/s eta 0:00:00


In [2]:
from openaq import OpenAQ
import fsspec
import json
import time

StatementMeta(, 035c5686-36d6-479b-b364-8240bbc7849c, 4, Finished, Available, Finished)

In [3]:
client = OpenAQ(api_key="d074d34fa5a6201fa2177fe98c5da21ec073be89871a5b7cb85e12584da0f253")

abfss_path = "abfss://4906b11e-1e59-4869-9321-062a4696a2db@onelake.dfs.fabric.microsoft.com/62794233-3c68-4109-ab1e-7666b1963827/Files/aq"
account_name = "4906b11e-1e59-4869-9321-062a4696a2db"
account_host = "onelake.dfs.fabric.microsoft.com"

fs = fsspec.filesystem(
    "abfss",
    account_name=account_name,
    account_host=account_host
)

StatementMeta(, 035c5686-36d6-479b-b364-8240bbc7849c, 5, Finished, Available, Finished)

In [4]:
page=1
ny_loc=[]
nyc_locations = {"CCNY", "Manhattan/IS143", "Bronx - IS52", "Maspeth", "Queens",
    "Bklyn - PS274", "Bronx - IS74", "Queens Near-road", "Morrisania"}
while True:
    response=client.locations.list(countries_id=155,page=page)
    result=response.results
    if not result:
        break
    for location in result:
        if location.name in nyc_locations:
            ny_loc.append(location.id)

    page+=1
    time.sleep(2)



time.sleep(60)


StatementMeta(, 035c5686-36d6-479b-b364-8240bbc7849c, 6, Finished, Available, Finished)

In [5]:

sensor_ids=[]
pollutants = {"pm25", "co", "no", "no2"}
for loc in ny_loc:
    sensors=client.locations.sensors(loc)
    for sensor in sensors.results:
        if sensor.parameter['name'] in pollutants:
            sensor_ids.append(sensor.id)
time.sleep(60)

StatementMeta(, 035c5686-36d6-479b-b364-8240bbc7849c, 7, Finished, Available, Finished)

In [6]:

sensor_metadata={}
for id in sensor_ids:
    sensor=client.sensors.get(id)
    temp=sensor.results[0].latest['coordinates']
    temp['last_fetch']=temp.get('last_fetch',None)
    sensor_metadata[id]=temp


StatementMeta(, 035c5686-36d6-479b-b364-8240bbc7849c, 8, Finished, Available, Finished)

In [7]:
sensor_folder = f"{abfss_path}/sensors"  
fs.makedirs(sensor_folder, exist_ok=True)
sensors_file=f"{sensor_folder}/sensors.json"
with fs.open(sensors_file,'w',encoding='utf-8') as file:
    json.dump(sensor_metadata,file,indent=2)

StatementMeta(, 035c5686-36d6-479b-b364-8240bbc7849c, 9, Finished, Available, Finished)